# Question 1: Time-series prediction






Prediction can be divided into two categories: regression and classification. In regression, the numerical value of the next days is predicted for a time series. In the classification, the upward or downward trend of the time series is predicted for the next few days.

In this question, you will learn about regression mode and implement a part of:
https://ieeexplore.ieee.org/document/9005997

## Import libraries

In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import datetime
from datetime import datetime
import yfinance as yf

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

## 1.1- Download data

In [9]:
table = pd.read_html('https://en.wikipedia.com/wiki/List_of_S%26P_500_companies')
table[0]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [6]:
New_table = table[0]

In [13]:
New_table_2 = New_table[[ datetime.strptime(dt, '%Y-%m-%d') < datetime(2010,1,1) for dt in New_table['Date added'] ] ]
New_table_2

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
5,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
7,AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,874761,1981
8,AFL,Aflac,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28,4977,1955
...,...,...,...,...,...,...,...,...
495,WYNN,Wynn Resorts,Consumer Discretionary,Casinos & Gaming,"Paradise, Nevada",2008-11-14,1174922,2002
496,XEL,Xcel Energy,Utilities,Multi-Utilities,"Minneapolis, Minnesota",1957-03-04,72903,1909
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [18]:
My_Tickers = New_table_2['Symbol'].tolist()
Raw_price_for_my_tickers = yf.download(tickers=My_Tickers, start="2010-01-04", interval="1d", group_by="ticker")


[*********************100%%**********************]  291 of 291 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2010-01-04 -> 2023-12-13)')


In [19]:
Raw_price_for_my_tickers

NTAP                                                       \
                 Open       High        Low      Close  Adj Close   Volume   
Date                                                                         
2010-01-04  34.590000  34.869999  34.000000  34.189999  26.354303  5320100   
2010-01-05  34.099998  34.349998  33.380001  33.590000  25.891825  7933900   
2010-01-06  33.470001  33.990002  33.099998  33.349998  25.706816  5193900   
2010-01-07  33.240002  33.529999  32.990002  33.270000  25.645161  5373000   
2010-01-08  33.119999  33.590000  32.779999  33.540001  25.853283  4718400   
...               ...        ...        ...        ...        ...      ...   
2023-12-06  91.000000  91.389999  89.230003  89.400002  89.400002  2282500   
2023-12-07  89.709999  90.489998  88.900002  89.080002  89.080002  2161200   
2023-12-08  89.080002  90.220001  88.930000  89.389999  89.389999  1633600   
2023-12-11  89.500000  90.599998  89.220001  89.750000  89.750000  2211600   
2023-12-12  90.000000  90.139999  88.000000  88.199997  88.199997  2131700   

                  AEP                                   ...         FE  \
                 Open       High        Low      Close  ...        Low   
Date                                                    ...              
2010-01-04  35.099998  36.000000  34.799999  34.939999  ...  46.410000   
2010-01-05  34.980000  35.119999  34.360001  34.540001  ...  45.669998   
2010-01-06  34.560001  35.099998  34.459999  34.889999  ...  46.020000   
2010-01-07  34.779999  35.310001  34.689999  35.189999  ...  45.980000   
2010-01-08  35.180000  35.810001  34.930000  35.610001  ...  45.529999   
...               ...        ...        ...        ...  ...        ...   
2023-12-06  79.959999  80.480003  79.570000  80.360001  ...  36.860001   
2023-12-07  80.500000  81.000000  79.830002  80.199997  ...  37.000000   
2023-12-08  80.120003  80.440002  79.230003  79.639999  ...  36.740002   
2023-12-11  79.820000  80.760002  79.510002  80.489998  ...  36.910000   
2023-12-12  80.599998  80.660004  79.599998  80.000000  ...  36.660000   

                                                 HAL                        \
                Close  Adj Close   Volume       Open       High        Low   
Date                                                                         
2010-01-04  46.770000  24.520954  1840900  30.719999  31.340000  30.660000   
2010-01-05  46.250000  24.248322  2922100  31.280001  31.830000  30.990000   
2010-01-06  46.750000  24.510471  3426300  31.740000  32.590000  31.639999   
2010-01-07  46.049999  24.143456  2327700  32.310001  32.520000  31.920000   
2010-01-08  45.810001  24.017639  2286900  32.950001  34.139999  32.490002   
...               ...        ...      ...        ...        ...        ...   
2023-12-06  37.450001  37.450001  3532700  35.889999  36.220001  34.790001   
2023-12-07  37.299999  37.299999  4221700  35.029999  35.259998  34.000000   
2023-12-08  37.080002  37.080002  4199600  34.750000  35.160000  34.400002   
2023-12-11  37.259998  37.259998  3762800  34.860001  35.439999  34.740002   
2023-12-12  36.980000  36.980000  3425200  34.200001  34.580002  33.790001   

                                            
                Close  Adj Close    Volume  
Date                                        
2010-01-04  31.250000  25.196728  11571600  
2010-01-05  31.650000  25.519236  18989700  
2010-01-06  32.400002  26.123972  15720800  
2010-01-07  32.480000  26.188467   8993100  
2010-01-08  34.119999  27.510796  23014600  
...               ...        ...       ...  
2023-12-06  34.860001  34.860001  10186300  
2023-12-07  34.419998  34.419998  11219300  
2023-12-08  34.930000  34.930000   6459200  
2023-12-11  34.770000  34.770000   6837600  
2023-12-12  34.439999  34.439999   8379100  

[3510 rows x 1746 columns]

## 2-1. Exploring time series data and familiarizing with famous theories and libraries

#### Question 1: If the null data was at the beginning, what methods can be used to solve this problem?

If we had null data at the beginning of a time series and we want to handle it, there are several methods we can consider:

* Forward Fill (Pad): This method involves filling missing values with the last known value. This approach assumes that the future values will be similar to the most recent past values.
` Raw_price_for_my_tickers.ffill(inplace=True) `

* Backward Fill (Bfill): This method involves filling missing values with the next available value. Similar to forward fill, it assumes that the past values will be similar to the most recent future values.
`Raw_price_for_my_tickers.bfill(inplace=True)`

* Interpolation: Interpolation is a method where missing values are estimated based on the values around them. There are different interpolation methods available, such as linear interpolation, polynomial interpolation, etc.
`Raw_price_for_my_tickers.interpolate(method='linear', inplace=True)`

* Replace with a Constant: You can replace with cons values.
`Raw_price_for_my_tickers.fillna(value=0, inplace=True)`

#### Question 2: If we have null value data in the middle or at the end of the time series, how do we solve it?


If we have null values in the middle or at the end of a time series, we can use similar methods to handle the missing data. Here are some common approaches:

There are several methods to handle missing values in time series data, including:

* Interpolation: This method replaces missing values with estimates based on the values of surrounding time points. Common interpolation methods include linear interpolation and spline interpolation.

* Extrapolation: This method extends the time series data beyond the existing data points by using mathematical models to estimate missing values.

* Forward-fill and backward-fill: In forward-fill, missing values are filled in with the next available value. In backward-fill, missing values are filled in with the previous available value.

* Last Observation Carried Forward (LOCF): This method replaces missing values with the last observed value.

* Multiple imputation: This method uses statistical methods to generate multiple estimates for missing values and then combines the results to produce a single estimate.

In [20]:
print("Missing values before imputation:")
print(Raw_price_for_my_tickers.isnull().sum())

Missing values before imputation:
NTAP  Open         0
      High         0
      Low          0
      Close        0
      Adj Close    0
                  ..
HAL   High         0
      Low          0
      Close        0
      Adj Close    0
      Volume       0
Length: 1746, dtype: int64


In [22]:
Raw_price_for_my_tickers.interpolate(method='linear', inplace=True)
print("\nMissing values after imputation:")
print(Raw_price_for_my_tickers.isnull().sum())


Missing values after imputation:
NTAP  Open         0
      High         0
      Low          0
      Close        0
      Adj Close    0
                  ..
HAL   High         0
      Low          0
      Close        0
      Adj Close    0
      Volume       0
Length: 1746, dtype: int64
